# Тема 2. Деревья принятия решений

## Agenda
В данном разделе вы познакомитесь с основными библиотеками python для анализа данных: pandas, matplotlib, sklearn и попробуете на практике провести анализ и сделать выводы по исходным данным.  
Целью данной лабораторной работы изучить принцип работы модели деревьев принятия решений, ее возможности.  
В качестве исходных данных был выбран датасет титаника (https://www.kaggle.com/c/titanic/data). 

## Дерево принятий решений
Дерево принятий решений - модель реализующая дерево, разделяющее каждый из объектов по заданным признакам. В ветках (ребрах) данного дерева находится условие разделения, а в листовых вершинах класс, к которому данное дерево относит объект. Пример такого дерева представлен на рисунке ниже

![tree_img](images/treeExample.PNG)

In [4]:
# Загрузим необходимы библиотеки
import pandas as pd
pd.set_option('display.max.columns', 100)
import matplotlib.pyplot as plt
import seaborn as sns

# Импортируем класс деревьев
from sklearn.tree import DecisionTreeClassifier
# И Функцию разделения на тестовую и обучающую выборку
from sklearn.model_selection import train_test_split

# Библиотека с математическими операциями над матрицами, 
# которая лежит в основе pandas
import numpy as np

# Для возможности вывода картинок прямо в тетрадь
%matplotlib inline 

In [5]:
# Загрузка дата сета
data = pd.read_csv("datasets/titanic_train.csv", index_col="PassengerId")

# Вывод на экран первых 5 записей
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# Создадим дубликат датасета без стоблца Cabin и удалим записи содержащие пустые ячейки
data_full = data.drop("Cabin", axis=1).dropna()

In [7]:
# Заменяем строку "male" на 0, "female" на 1
data_full["Gender"] = data_full["Sex"].map({"male": 0, "female": 1})
data_full["Gender"].head()

PassengerId
1    0
2    1
3    1
4    1
5    0
Name: Gender, dtype: int64

In [8]:
data_full.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Gender
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0


In [9]:
# Выделим отдельно стобцы на основе которых будем строить модель
X = data_full[["Age", "Gender"]]
# Выделим целевую переменную
y = data_full.Survived

In [10]:
# Разделим выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=40,
                                                   shuffle=True)

Для начала попробуем без настройки гиперпараметров обучить дерево

In [19]:
# Создадим модель и установим фиксированный random_state для воспроизводимости результатов
model = DecisionTreeClassifier(random_state=42)

In [20]:
# Обучим модель
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [21]:
# Проверим долю правильных ответов нашей модели на тестовой выборке
acc = np.sum(model.predict(X_test) == y_test) / X_test.shape[0]
print("Доля правильных ответов = {}".format(acc))

Доля правильных ответов = 0.7132867132867133


Как мы видим с 71% точностью наша модель правильно определяет выживаемость того или иного пассажира титаника

Попробуем улучшить нашу модель. Добавим 3 новых признака: первый класс, второй класс и третий класс. Эти данные можно взять из столбца "Pclass".
Для разбития качественного признака в несколько бинарных сущесвует метод **pd.get_dummies**

In [22]:
data_full = pd.concat([data_full, pd.get_dummies(data_full["Pclass"], prefix="Pclass")], axis=1)
data_full.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Gender,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,0,0,0,1
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,1,1,0,0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,1,0,0,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,1,1,0,0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,0,0,0,1


In [23]:
# Выделим отдельно стобцы на основе которых будем строить модель
selected_cols = ["Pclass_1", "Pclass_2", "Pclass_3", "Age", "Gender"]
X = data_full[selected_cols]
# Выделим целевую переменную
y = data_full.Survived

In [50]:
# Разделим выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, random_state=40,
                                                   shuffle=True)

In [51]:
# Создадим модель и установим фиксированный random_state для воспроизводимости результатов
model = DecisionTreeClassifier(random_state=42)
# Обучим модель
model.fit(X_train, y_train)
# Проверим долю правильных ответов нашей модели на тестовой выборке
acc = np.sum(model.predict(X_test) == y_test) / X_test.shape[0]
print("Доля правильных ответов = {}".format(acc))

Доля правильных ответов = 0.7762237762237763


In [52]:
# Выведем признаки и их важности в модели
for i, col_name in enumerate(selected_cols):
    print("{} = {}".format(col_name, model.feature_importances_[i]))

Pclass_1 = 0.03848726745272861
Pclass_2 = 0.011238558869084689
Pclass_3 = 0.13323160379773744
Age = 0.4079201756465267
Gender = 0.40912239423392255


Как видно, наибольшее влияние на выживаемость влияет возраст и пол. Теперь попробуем подобрать гиперпараметры, чтобы доля правильных ответов на тестовой выборке повысилась. Основным гиперпараметром дерева является его глубина. При большой глубине дерева, оно может переобучиться и выдавать плохие результаты на тестовой выборке, но отличные на обучающей. 

In [16]:
# Попробуем построить деревья с максимальной глубиной от 2 до 15
for depth in range(2, 15):
    # Создадим модель и установим фиксированный random_state для воспроизводимости результатов
    model = DecisionTreeClassifier(random_state=42, max_depth=depth)
    # Обучим модель
    model.fit(X_train, y_train)
    # Проверим долю правильных ответов нашей модели на тестовой выборке
    acc = np.sum(model.predict(X_test) == y_test) / X_test.shape[0]
    print("Доля правильных ответов = {} при глубине равной = {}".format(acc, depth))

Доля правильных ответов = 0.7622377622377622 при глубине равной = 2
Доля правильных ответов = 0.7762237762237763 при глубине равной = 3
Доля правильных ответов = 0.7762237762237763 при глубине равной = 4
Доля правильных ответов = 0.7622377622377622 при глубине равной = 5
Доля правильных ответов = 0.7762237762237763 при глубине равной = 6
Доля правильных ответов = 0.7762237762237763 при глубине равной = 7
Доля правильных ответов = 0.7762237762237763 при глубине равной = 8
Доля правильных ответов = 0.7832167832167832 при глубине равной = 9
Доля правильных ответов = 0.7692307692307693 при глубине равной = 10
Доля правильных ответов = 0.7692307692307693 при глубине равной = 11
Доля правильных ответов = 0.7762237762237763 при глубине равной = 12
Доля правильных ответов = 0.7762237762237763 при глубине равной = 13
Доля правильных ответов = 0.7762237762237763 при глубине равной = 14


Как видно, при глубине равно 9 достигается максимальная доля правильных ответов равной 0.783. Это значение является наиболее подходящим для данной модели

## Случайный лес

В основе случайного леса лежит теорема Кондорсе «о жюри присяжных». Если каждый присяжный выносит правильный вердикт с вероятностью больше 50%, то усредненный вердикт присяжных будет иметь большую вероятность истиности.  
Случайные леса в своей основе используют деревья принятий решений и алгоритм обучения модели состоит из следующих шагов:  
1. Разбить обучающую выборку на n частей с замещением
2. Создать n деревьев принятий решений и обучить каждый на своей части обучающей выборки
3. Получить предсказания каждого дерева для тестовой выборки и усреднить

Случайный лес реализован в классе sklearn.ensemble.RandomForestClassifier. Основными гиперпараметрами для настройки деревьев являеются n_estimators, которое определяет количество деревьев и max_depth определяющих их глубину.

In [3]:
# Импортируем
from sklearn.ensemble import RandomForestClassifier

In [75]:
# Создадим лес с 100 деревьев
model = RandomForestClassifier(n_estimators=20, random_state=42)

In [76]:
# Обучим модель
model.fit(X_train, y_train)
# Проверим долю правильных ответов нашей модели на тестовой выборке
acc = np.sum(model.predict(X_test) == y_test) / X_test.shape[0]
print("Доля правильных ответов = {}".format(acc))

Доля правильных ответов = 0.7762237762237763


Как видим результат уже получился точно такой же как и при использовании одного дерева. Попробуем настроить гиперпараметры модели. 

In [77]:
best_val = 0
best_model = None
for n in range(1, 20, 1):
    for depth in range(2, 20):
        # Создадим лес с 100 деревьев
        model = RandomForestClassifier(n_estimators=n, max_depth=depth, random_state=42)
        # Обучим модель
        model.fit(X_train, y_train)
        # Проверим долю правильных ответов нашей модели на тестовой выборке
        acc = np.sum(model.predict(X_test) == y_test) / X_test.shape[0]
        if (acc > best_val):
            best_val = acc
            best_model = model

In [78]:
print("Best score = {}".format(best_val))

Best score = 0.8181818181818182


Как видим, в данном случае получился более точный результат, 81.8%, который гораздо лучше чем был при использовании одного дерева - 78.3%

In [79]:
print("Best params:\n Depth = {}\n Num of estimators = {}".format(best_model.max_depth, best_model.n_estimators))

Best params:
 Depth = 5
 Num of estimators = 18


## Задание на лабораторную
Файлы с исходными данными находятся по пути *Datasets/classification/v<номер варианта>.csv*

| Вариант | Целевая переменная | Краткое описание | Источник
|-------|------------|----------------------------|-------------------|
|1|y|Предсказать, воспользуется ли клиент временным депозитом.|https://archive.ics.uci.edu/ml/datasets/bank+marketing
|2|bad|Предсказать значение переменной BAD, показывающей что клиент банка не выполнит свои обязательства(например не выплатит кредит).|https://www.kaggle.com/ajay1735/hmeq-data/data
|3|churn|Предсказать что клинет оператора(столбец churn) откажется от его услуг|https://www.kaggle.com/becksddf/churn-in-telecoms-dataset/data
|4|RainTomorrow|Предсказать, будет ли завтра дождь(Столбец RainTomorrow), на основе данные за текущий день |https://www.kaggle.com/jsphyg/weather-dataset-rattle-package
|5|sol|Пресказать, зарабатывает ли человек более 50k долларов в год|https://archive.ics.uci.edu/ml/datasets/Adult
|6|acceptable|Предсказать, что качество машины в удовлетворительном состоянии( столбец acceptable)|https://archive.ics.uci.edu/ml/datasets/Car+Evaluation
|7|Severity| Предсказать тип образование ( столбец Severity, 1 - злокачественная) |https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass
|8|dicision|Определить, готов ли пациент для выписки из больницы( столбец dicision = 1)|https://archive.ics.uci.edu/ml/datasets/Post-Operative+Patient
|9|edible|Определить, является ли гриб съедобным по его внешним признакам( столбец edible)|https://archive.ics.uci.edu/ml/datasets/Mushroom
|10|class|Предсказать банкротство на основе рисков( столбец class)|https://archive.ics.uci.edu/ml/datasets/Qualitative_Bankruptcy
|11|class|Определить каким типом посадки необходимо воспользоваться : автоматическая или ручная ( столбец class)|https://archive.ics.uci.edu/ml/datasets/Shuttle+Landing+Control
|12|absence|Определить наличие болезни сердца ( столбец absence )|https://archive.ics.uci.edu/ml/datasets/Statlog+%28Heart%29
|13|target|Используя данные о сессии браузера, определить является пользователем Элис Целевой признак target – факт того, что сессия принадлжит Элис (то есть что именно Элис ходила по всем этим сайтам)|https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking
|14|survive|Определить, проживёт ли пациент более 5 лет после проведения операции|https://archive.ics.uci.edu/ml/datasets/Haberman%27s+Survival
|15|isAlive|Определить, выживет ли персонаж игры престолов(колонка isAlive)|https://www.kaggle.com/mylesoneill/game-of-thrones/data